In [1]:
import sys
import numpy as np
import cvxpy as cp
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
import pmdarima
from dateutil import parser
import json

from case_1 import case_1
from case_2 import case_2
from case_3 import case_3
from case_4 import case_4

from case_1 import plot_case_1
from case_2 import plot_case_2
from case_3 import plot_case_3
from case_4 import plot_case_4

from plot_case_1_3 import plot_case_1_3


class MPC(object):
    def __init__(self, data, temp_data, filename):
        self.data_list = data
        self.temp_data = temp_data
        params = json.load(open(filename))

        self.eta = params['eta']
        self.batt_energy = params['batt_energy'] # kWh
        self.chg_power = params['chg_power']  #kW
        self.dischg_power = params['dischg_power'] # kW
        self.transformer_limit = params['transformer_limit'] # kW
        self.case = params['case']
        self.num_homes = params['num_homes']
        self.soc_init = params['soc_init']
        self.lam = params['lambda']

        self.dt = params['dt']
        self.total_horizon = params['total_horizon']
        self.mpc_horizon = params['mpc_horizon']
        self.num_mpc_iters = int(self.total_horizon  / self.dt)
        self.forecast_days = params['forecast_days']
        self.num_timesteps = int(self.mpc_horizon / self.dt)

        #Austin TOU
        self.off_peak = 0.02841
        self.partial_peak = 0.04371
        self.peak = 0.08961

        self.colorset = ['#4477AA', '#EE6677', '#228833', '#CCBB44', '#66CCEE', '#AA3377', '#BBBBBB']
        
        
        #average out data to fit MPC - downsampling only!
        for home in range(self.num_homes):
            temp = pd.DataFrame(columns = ['datetime', 'solar', 'loads'])
            self.data_temp = self.data_list[home]
            current_dt = (self.data_temp.datetime[1].minute - self.data_temp.datetime[0].minute) / 60
            sampling_rate = int(self.dt/current_dt)
            temp.datetime = self.data_temp.datetime[::sampling_rate]
            
            temp.solar  = np.average(np.array(self.data_temp.solar).reshape(-1, sampling_rate), axis=1)
            temp.loads = np.average(np.array(self.data_temp.loads).reshape(-1, sampling_rate), axis=1)
            
            self.data_temp = temp
            self.data_temp = self.data_temp.reset_index(drop=True)
            self.data_list[home] = self.data_temp
        
        


    def run_mpc(self, solar, loads, timesteps, num_homes, time_vec, soc_init = .5):

        if self.case ==1:
            meter, batt_output, costs, SOCs = case_1(self, solar, loads, timesteps, num_homes, time_vec, soc_init)
        if self.case ==2:
            meter, batt_output, costs, SOCs = case_2(self, solar, loads, timesteps, num_homes, time_vec, soc_init)
        if self.case ==3:
            meter, batt_output, costs, SOCs = case_3(self, solar, loads, timesteps, num_homes, time_vec, soc_init)
        if self.case ==4:
            meter, batt_output, costs, SOCs = case_4(self, solar, loads, timesteps, num_homes, time_vec, soc_init)

        return meter, batt_output, costs, SOCs
    
    def take_mpc_step(self, soc_old, batt_output, time):
        soc_new = soc_old - self.dt * batt_output / self.batt_energy
        solar_actual = np.zeros((self.num_homes,))
        loads_actual = np.zeros((self.num_homes,))
        for home in range(self.num_homes):
            solar_actual[home] = self.data_list[home].loc[np.where(self.data_list[home].loc[:, 'datetime']==time)[0][0], 'solar']
            loads_actual[home] = self.data_list[home].loc[np.where(self.data_list[home].loc[:, 'datetime']==time)[0][0], 'loads']
        meter = -solar_actual + loads_actual - batt_output
        
        return soc_new, meter
    
    def calc_mpc_cost(self, meter, num_homes, time_vec, batt_output=None, no_bess_val=False):
        timesteps = len(time_vec)
        price_vector = np.zeros((timesteps,))
        for time in range(timesteps):
            # weekend - off peak
            if time_vec[time].weekday() == 5 or time_vec[time].weekday() == 6:
                price_vector[time] = self.off_peak
            else: #weekday
                if time_vec[time].hour < 7 or time_vec[time].hour >=22: #off peak
                    price_vector[time] = self.off_peak
                elif time_vec[time].hour>=15 and time_vec[time].hour<18: #peak
                    price_vector[time] = self.peak
                else: #off peak
                    price_vector[time] = self.partial_peak

        if no_bess_val:
            cost_vec = np.zeros((num_homes,))
            for home in range(num_homes):
                cost_vec[home]= np.maximum(meter[:, home],0) * self.dt @ price_vector
        elif self.case ==1:
            cost_vec = np.zeros((num_homes,))
            for home in range(num_homes):
                cost_vec[home]= np.maximum(meter[:, home] -  batt_output[:, home], 0) * self.dt @ price_vector

        elif self.case==3 or self.case==4:
            combined_cost =  np.maximum(np.sum(meter, axis=1) -  batt_output[:, 0],0 ) * self.dt @ price_vector
            cost_vec = np.zeros((num_homes,))
            orig_costs = np.zeros((num_homes,), dtype='float32')
            for home in range(num_homes):
                orig_costs[home] = np.maximum(meter[:, home],0) * self.dt @ price_vector
        
            for home in range(num_homes):
                cost_vec[home] = orig_costs[home] / np.sum(orig_costs) * combined_cost
        

        return cost_vec

    def getDates(self, startDate):
        for timestep in range(self.num_timesteps):
            yield [startDate - datetime.timedelta(days  = self.forecast_days) + \
                               datetime.timedelta(hours = timestep * self.dt) + \
                               datetime.timedelta(days  = day) for day in range(self.forecast_days)]
    
    def forecastPlots(self, forecast_df, f, f2, start):
        
            
        dates = list(self.getDates(start))
        if self.num_homes>1:
            fig, axes = plt.subplots(self.num_homes,1)
            for home in range(self.num_homes):
                count=0
                for currentDay in np.array(dates).transpose():
                    
                    solar = self.data_list[home].loc[np.isin(self.data_list[home].datetime, currentDay), 'solar']
                    axes[home].plot(forecast_df.datetime, solar, color=self.colorset[count+2], label='Day '+ str(count))
                    count+=1

                times  = [d[0] + datetime.timedelta(days = self.forecast_days) for d in dates]
                currDF = self.data_list[home][np.isin(self.data_list[home].datetime, times)]
                axes[home].plot(forecast_df.datetime, f[home]["solar_mean"],color=self.colorset[1],  label = 'avg')
                axes[home].plot(currDF.datetime, currDF.solar, color=self.colorset[0], label = 'actual')
                axes[home].set_ylabel('Power [kw]')
                axes[home].legend()


            # Plot Solar
            fig, axes = plt.subplots(self.num_homes,1)

            times  = [d[0] + datetime.timedelta(days = self.forecast_days) for d in dates]
            currDF = self.data_list[0][np.isin(self.data_list[0].datetime, times)]

            for home in range(self.num_homes):
                currDF = self.data_list[home][np.isin(self.data_list[home].datetime, times)]
                axes[home].plot(currDF.datetime, currDF.solar, color=self.colorset[0], label = 'actual')
                axes[home].plot(forecast_df.datetime, f[home]["solar_mean"], color=self.colorset[1], label = "avg")
                axes[home].set_ylabel('Power [kw]')
                axes[home].legend()
            plt.title('Solar')
            plt.show()

            # Plot Loads 
            fig, axes = plt.subplots(self.num_homes,1)
            for home in range(self.num_homes):
                currDF = self.data_list[home][np.isin(self.data_list[0].datetime, times)]
                axes[home].plot(currDF.datetime, currDF.loads,color=self.colorset[0], label = 'Actual')
                axes[home].plot(forecast_df.datetime, f[home]["loads_mean"],color=self.colorset[1], label = "Prediction")
                axes[home].set_ylabel('Power [kw]')
                axes[home].legend()
            plt.title('Loads')
            plt.show()

            fig, axes = plt.subplots(2,1, figsize=(6,6))
            currDF = self.data_list[0][np.isin(self.data_list[0].datetime, times)]
            axes[0].plot(np.arange(len(currDF.datetime)), currDF.solar,color='0', label = 'Actual Solar', linewidth=2)
            axes[0].plot(np.arange(len(forecast_df.datetime)), f[0]["solar_mean"],color=self.colorset[1], label = 'Forecast Solar', linewidth=2)
            #axes[0].plot(np.arange(len(forecast_df.datetime)), f2[0]["solar_mean"],color=self.colorset[2], label = 'Conservative Forecast Solar', linewidth=2)

            axes[0].set_ylabel('Power [kW]', fontsize=14)

            axes[0].legend(fontsize=11, loc=1)
            axes[0].set_xticks(range(0,len(forecast_df.datetime)+1,8))
            axes[0].set_xticklabels(range(0,24+1,4))
            axes[0].tick_params(axis='both', which='major', labelsize=11)
            #axes[0].xaxis.set_tick_params(labelbottom=False)
            currDF = self.data_list[0][np.isin(self.data_list[0].datetime, times)]
            axes[1].plot(np.arange(len(currDF.datetime)), currDF.loads,color='0', label = 'Actual Loads', linewidth=2)
            axes[1].plot(np.arange(len(forecast_df.datetime)), f[0]["loads_mean"], color=self.colorset[1], label = "Forecast Loads", linewidth=2)
            #axes[1].plot(np.arange(len(forecast_df.datetime)), f2[0]["loads_mean"], color=self.colorset[2], label = "Conservative Forecast Loads", linewidth=2)

            axes[1].set_ylabel('Power [kW]', fontsize=14)
            axes[1].legend(fontsize=11, loc=1)
            axes[1].set_xlabel('Time [hr]', fontsize=14)
            axes[1].set_xticks(range(0,len(forecast_df.datetime)+1,8))
            axes[1].set_xticklabels(range(0,24+1,4))
            axes[1].set_ylim([0,max(currDF.loads)*1.1])
            axes[1].tick_params(axis='both', which='major', labelsize=11)
            #plt.setp(axes, xticks=range(0,len(forecast_df.datetime),8), xticklabels=range(0,24,4))
            plt.savefig('Figures/forecast.pdf', bbox_inches='tight')
            plt.savefig('Figures/forecast.png', bbox_inches='tight')
            plt.show()

            print('Day = ' + str(start))
            solar_mape= []
            loads_mape = []
            for home in range(self.num_homes):
                solar_mape.append(abs((sum(self.data_list[home][np.isin(self.data_list[0].datetime, times)].solar)-sum(f[home]["solar_mean"]))/sum(self.data_list[0][np.isin(self.data_list[0].datetime, times)].solar)))
                loads_mape.append(abs((sum(self.data_list[home][np.isin(self.data_list[0].datetime, times)].loads)-sum(f[home]["loads_mean"]))/sum(self.data_list[0][np.isin(self.data_list[0].datetime, times)].loads)))
            solar_mape_final = (1/self.num_homes) * sum(solar_mape)
            loads_mape_final = (1/self.num_homes) * sum(loads_mape)
            print('Solar MAPE = ' + str(solar_mape_final))
            print('Loads MAPE = ' + str(loads_mape_final))


        else:
            fig, axes = plt.subplots(self.num_homes,1)
            for currentDay in np.array(dates).transpose():
                solar = self.data_list[0][np.isin(self.data_list[0].datetime, currentDay)].solar
                axes.plot(forecast_df.datetime, solar)

            axes.plot(forecast_df.datetime, f[0]["solar_mean"], color=self.colorset[1], label = 'avg')
            axes.set_ylabel('Power [kw]')
            axes.legend()
            
            plt.show()

            fig, axes = plt.subplots(self.num_homes,1)

            times  = [d[0] + datetime.timedelta(days = self.forecast_days) for d in dates]
            currDF = self.data_list[0][np.isin(self.data_list[0].datetime, times)]

            
            axes.plot(currDF.datetime, currDF.solar, color=self.colorset[0], label = 'actual')
            axes.plot(forecast_df.datetime, f[0]["solar_mean"], color=self.colorset[1], label = "avg")
            axes.set_ylabel('Power [kw]')
            axes.legend()
            plt.title('Solar')
            plt.show()

            fig, axes = plt.subplots(self.num_homes,1)
            
            axes.plot(currDF.datetime, currDF.loads, color=self.colorset[0], label = 'Actual')
            axes.plot(forecast_df.datetime, f[0]["loads_mean"], color=self.colorset[1], label = "Prediction")
            axes.set_ylabel('Power [kw]')
            axes.legend()
            plt.title('Loads')
            plt.show()
        
        return
    
    def return_forecast(self, start, plotting=False, forecast_type=1):
        forecast_df = pd.DataFrame(columns = ['datetime'])
        forecast_df.loc[0, 'datetime'] = start 

        for timestep in range(1, self.num_timesteps):
            forecast_df.loc[timestep, 'datetime'] = forecast_df.loc[0, 'datetime'] + timestep * datetime.timedelta(hours=self.dt)

        f = [{"solar_mean" : [], 
              "loads_mean" : []} for home in range(self.num_homes)]

        if forecast_type==1:
            for timestep in range(self.num_timesteps):
                avg_idxs = []
                for day in range(self.forecast_days):
                    avg_idxs.append(np.where(self.data_list[0].loc[:, 'datetime'] == start - datetime.timedelta(days=self.forecast_days) + datetime.timedelta(hours = timestep * self.dt) + datetime.timedelta(days=day))[0][0])
                
                for home in range(self.num_homes):

                    f[home]["solar_mean"].append(np.average(self.data_list[home].loc[avg_idxs, 'solar']))
                    f[home]["loads_mean"].append(np.average(self.data_list[home].loc[avg_idxs, 'loads']))
                
        

        if forecast_type==2:
            for timestep in range(self.num_timesteps):
                avg_idxs = []
                for day in range(self.forecast_days):
                    avg_idxs.append(np.where(self.data_list[0].loc[:, 'datetime'] == start - datetime.timedelta(days=self.forecast_days) + datetime.timedelta(hours = timestep * self.dt) + datetime.timedelta(days=day))[0][0])
                
                for home in range(self.num_homes):

                    f[home]["solar_mean"].append(np.average(self.data_list[home].loc[avg_idxs, 'solar']) * 0.8)
                    f[home]["loads_mean"].append(np.average(self.data_list[home].loc[avg_idxs, 'loads']) * 1.2)
            
        
        solar = np.zeros((len(forecast_df.datetime), self.num_homes))
        loads = np.zeros((len(forecast_df.datetime), self.num_homes))

        for home in range(self.num_homes):
            solar[:, home] = f[home]["solar_mean"] 
            loads[:, home] = f[home]["loads_mean"] 

        
        if plotting:
            # a bit of plotting
            f2 = [{"solar_mean" : [], 
              "loads_mean" : []} for home in range(self.num_homes)]
            for timestep in range(self.num_timesteps):
                avg_idxs = []
                for day in range(self.forecast_days):
                    avg_idxs.append(np.where(self.data_list[0].loc[:, 'datetime'] == start - datetime.timedelta(days=self.forecast_days) + datetime.timedelta(hours = timestep * self.dt) + datetime.timedelta(days=day))[0][0])
                
                for home in range(self.num_homes):

                    f2[home]["solar_mean"].append(np.average(self.data_list[home].loc[avg_idxs, 'solar']) * 0.8)
                    f2[home]["loads_mean"].append(np.average(self.data_list[home].loc[avg_idxs, 'loads']) * 1.2)
            self.forecastPlots(forecast_df, f, f2, start)

        return forecast_df.datetime, solar, loads


    def deterministic_mpc(self, start_date, plotting = False, forecast = 1):

        meter       = np.zeros((self.num_mpc_iters, self.num_homes))
        batt_output = np.zeros((self.num_mpc_iters, self.num_homes))
        soc_init    = np.zeros((self.num_mpc_iters+1, self.num_homes))

        soc_init[0, :] = self.soc_init
        batt_vec_list  = []

        for mpc_iter in range(self.num_mpc_iters):
            print(str(mpc_iter+1) + ' iteration out of ' + str(self.num_mpc_iters) + ' MPC iterations')
            time = start_date + datetime.timedelta(hours = mpc_iter * self.dt)
            
            dates, solar, loads = self.return_forecast(start_date + datetime.timedelta(hours = mpc_iter * self.dt), plotting = False, forecast_type = forecast)
            
            m, b, cost_vec, soc = self.run_mpc(solar, loads, self.num_timesteps, self.num_homes, dates, soc_init = soc_init[mpc_iter, :])
            batt_output[mpc_iter, :] = b[0, :]
            soc_init[mpc_iter+1, :], meter[mpc_iter, :] = self.take_mpc_step(soc_init[mpc_iter, :], batt_output[mpc_iter, :], time)
            batt_vec_list.append(b)

        if plotting:
            start_idx = np.where(self.data_list[0].loc[:, 'datetime']==start_date)[0][0]
            end_idx = np.where(self.data_list[0].loc[:, 'datetime']==start_date + datetime.timedelta(hours = self.total_horizon-self.dt) )[0][0] 
            fig, axes = plt.subplots(int(b.shape[1]),1)
            
            if b.shape[1]>1:
                for home in range(b.shape[1]):
                    for iter in range(0,len(batt_vec_list),4):
                        date = self.data_list[0].loc[start_idx+iter:np.minimum(end_idx,start_idx+iter+self.num_timesteps-1), 'datetime'].values
                        axes[home].plot(date, batt_vec_list[iter][:len(date), home], color = self.colorset[1])
                    axes[home].plot(self.data_list[0].loc[start_idx:end_idx, 'datetime'].values, batt_output[:, home], label = 'Actual Output', color = self.colorset[0])
                    axes[home].set_ylabel('Power [kW]')
            else:
                for iter in range(0,len(batt_vec_list),4):
                    date = self.data_list[0].loc[start_idx+iter:np.minimum(end_idx,start_idx+iter+self.num_timesteps-1), 'datetime'].values
                    axes.plot(date, batt_vec_list[iter][:len(date)], color = self.colorset[1])
                axes.plot(self.data_list[0].loc[start_idx:end_idx, 'datetime'].values, batt_output, label = 'Actual Output', color = self.colorset[0])
                axes.set_ylabel('Power [kW]')

            plt.xlabel('Date')
            
            plt.legend()
            plt.show()

        start_idx = np.where(self.data_list[0].loc[:, 'datetime']==start_date)[0][0]
        end_idx = np.where(self.data_list[0].loc[:, 'datetime']==start_date + datetime.timedelta(hours = self.total_horizon - self.dt))[0][0] 
        time_vec = self.data_list[0].loc[start_idx:end_idx, 'datetime'].values
        

        no_bess = np.zeros((len(time_vec), self.num_homes))
        for home in range(self.num_homes):
            no_bess[:, home] = self.data_list[home].loc[start_idx:end_idx, 'loads'].values - self.data_list[home].loc[start_idx:end_idx, 'solar'].values

        cost_vec = self.calc_mpc_cost(no_bess, self.num_homes, time_vec, batt_output)

        print('Total MPC cost is: ', np.sum(cost_vec))
        print('cost of each home is: ', cost_vec)
        
        if self.num_homes >1:
            cost_vec_no_bess = self.calc_mpc_cost(no_bess, self.num_homes, time_vec, batt_output=None, no_bess_val=True)
        else:
            cost_vec_no_bess = self.calc_mpc_cost(no_bess.reshape(-1,1), self.num_homes, time_vec, batt_output=None, no_bess_val=True)

        print('Total cost without BESS: ', np.sum(cost_vec_no_bess))
        print('cost of each home is: ', cost_vec_no_bess)


        return meter, batt_output, soc_init, cost_vec, cost_vec_no_bess
    
    def full_deterministic(self, start_date, plotting=False):
        solar     = np.zeros((self.num_mpc_iters, self.num_homes))
        loads     = np.zeros((self.num_mpc_iters, self.num_homes))
        soc_init  = np.zeros((self.num_homes,))

        
        soc_init[:] = self.soc_init

        for home in range(self.num_homes):
            mask = self.data_list[home][(self.data_list[home].datetime >= start_date) & (self.data_list[home].datetime <  start_date + datetime.timedelta(hours = self.total_horizon))].index
            solar[:, home] = self.data_list[home].loc[mask, 'solar']
            loads[:, home] = self.data_list[home].loc[mask, 'loads']
            time_vec    = self.data_list[home].loc[mask].datetime.values
 
        meter_amount, batt_output, cost_vec, soc_output = self.run_mpc(solar, loads, self.num_mpc_iters, self.num_homes, time_vec, soc_init)
        
        print('Total full deterministic cost is: ', np.sum(cost_vec))
        print('cost of each home is: ', cost_vec)

        return meter_amount, batt_output, soc_output, cost_vec

    
    
def process_data(filename=None):
    if filename == None:
        data = pd.read_csv('Data/house_9836.csv')
    else:
        data=pd.read_csv(filename)
    #data = data.rename(columns={'Date & Time':'datetime', 'Solar [kW]':'solar', 'Grid [kW]':'loads', 'EV Loads [kW]':'ev'})
    data = data.rename(columns={'loads [kW]':'loads', 'solar [kW]':'solar', 'ev (kW)':'ev'})
    #data = data.rename(columns={'Date & Time':'datetime', 'Solar [kW]':'solar', 'EV Loads[kW]':'ev'})
    #data.loads += data.solar
    
    #data.datetime = pd.to_datetime(data.datetime)
    #data cleaning:
    data.loc[np.where(data.solar < 0)[0][:], 'solar'] = 0
    data.loc[np.where(data.ev < 0.01)[0][:], 'ev'] = 0

    #data.solar[data.solar < 0] = 0
    #data.ev[data.ev < 0.01] = 0

    #add EV data to loads - if not already pre-processed
    data.loads += data.ev


    for i in range(len(data.datetime)):
        data.loc[i, 'datetime'] = parser.parse(str(data.loc[i,'datetime'])) # datetime.datetime.strptime(str(data.loc[i,'datetime']), '%Y-%m-%d %H:%M:%S')
        data.loc[i, 'datetime'] =  datetime.datetime.replace(data.loc[i, 'datetime'], tzinfo=None)
    return data

def process_temp_data(dt):
    temp_input = pd.read_csv('../Data/austin_temp_2018.csv')
    temp_df = pd.DataFrame(columns = ['datetime', 'temperature'])
    num_step = int(1/dt)
    temp_df['temperature'] = np.repeat(temp_input.iloc[:,2], num_step)
    temp_df['datetime'] = np.repeat(temp_input.iloc[:,0], num_step)
    temp_df = temp_df.reset_index(drop=True)
    

    #add hours + minutes
    for i in range(len(temp_input)):
        for j in range(num_step):
            temp_df.loc[i*num_step+j, 'datetime'] = datetime.datetime.strptime(str(temp_df.loc[i*num_step+j,'datetime']), '%Y%m%d')+ datetime.timedelta(hours = temp_input.iloc[i,1]/100) + datetime.timedelta(hours =dt * j)
    return temp_df

def open_saved_data(path):
    meter_d_mpc = np.loadtxt(path + '/meter_d_mpc.csv')
    batt_output_d_mpc = np.loadtxt(path + '/batt_output_d_mpc.csv')
    soc_d_mpc = np.loadtxt(path + '/soc_d_mpc.csv')
    meter_d = np.loadtxt(path + '/meter_d.csv')
    batt_output_d = np.loadtxt(path + '/batt_output_d.csv')
    soc_d = np.loadtxt(path + '/soc_d.csv')
    return meter_d_mpc, batt_output_d_mpc, soc_d_mpc,meter_d, batt_output_d, soc_d

def cost_savings(no_bess, indiv, joint):
    indiv_save = []
    joint_save = []
    for home in range(len(no_bess)):
        indiv_save.append((no_bess[home] - indiv[home])/no_bess[home])
        joint_save.append((no_bess[home] - joint[home])/no_bess[home])

    print('indiv savings: ', np.average(indiv_save))
    print('joint savings: ', np.average(joint_save))
    return np.average(indiv_save), np.average(joint_save)


def main():
    # if len(sys.argv) != 2:
    #     sys.exit("./optimization.py inputFileName")
    #solar, loads, data = process_data(sys.argv[1])
    temp_data = process_temp_data(5/60)
    #homeIDs   = [661, 4373, 4767, 6139, 7719, 8156, 1642, 8156, 1642]
    #createFileName = lambda start,end,ID: "Data_L2/data_{}_jan_{}_{}_l2.csv".format(ID, start, end)
    month = 1
    start_day = 4
    #filenames = [createFileName(start_day, start_day + 10, ID) for ID in homeIDs]
    #filenames = [createFileName(start_day, start_day + 17, ID) for ID in homeIDs]

    IDs = [102808, 108199, 122340, 131172, 138546, 146132, 153948, 165637, 180477]
    createFileName = lambda ID : "../Data_L2/csv/{}-0.csv".format(ID)
    filenames = [createFileName(ID) for ID in IDs]

    data = []


    for home_file in filenames:
        data.append(process_data(home_file))

    #experiment_file = 'Code/Experiments/case1_2.json'
    #experiment_file = 'Code/Experiments/case1_1.json'


    #experiment_file = 'Code/Experiments/case1_2.json'
    #experiment_file = 'Code/Experiments/case1_7_homes.json'
    experiment_file = '../Code/Experiments/case1_9_homes.json'
    #experiment_file = 'Code/Experiments/case1_9_homes_2wks.json'
    

    start_date = datetime.datetime(2018, month, start_day + 4, 0 ,0)
    mpc_problem = MPC(data, temp_data, experiment_file)

    #path = 'Code/Results/case_1_9_homes_l2'
    #for day in range(int(mpc_problem.total_horizon/mpc_problem.mpc_horizon)):
    #    mpc_problem.return_forecast(start_date+datetime.timedelta(days=day), plotting=True)

    mpc_problem.return_forecast(start_date+datetime.timedelta(days=3), plotting=True)
    
    

    # to run case 1

    #forecast 1
    path = '../Code/Results/case_1_9_homes_l2/f1'
    #path = 'Code/Results/case_1_9_homes_l2/two_weeks'
    meter_d_mpc, batt_output_d_mpc, soc_d_mpc, cost_list_mpc_1, cost_list_1_no_bess = mpc_problem.deterministic_mpc(start_date, plotting=True, forecast=1)
    meter_d, batt_output_d, soc_d, cost_list_determ_1 = mpc_problem.full_deterministic(start_date)
    np.savetxt(path + '/meter_d_mpc.csv', meter_d_mpc)
    np.savetxt(path + '/batt_output_d_mpc.csv', batt_output_d_mpc)
    np.savetxt(path + '/soc_d_mpc.csv', soc_d_mpc)
    np.savetxt(path + '/meter_d.csv', meter_d)
    np.savetxt(path + '/batt_output_d.csv', batt_output_d)
    np.savetxt(path + '/soc_d.csv', soc_d)
    np.savetxt(path + '/cost_list_determ_1.csv', cost_list_determ_1)
    np.savetxt(path + '/cost_list_mpc_1.csv', cost_list_mpc_1)

    meter_d_mpc, batt_output_d_mpc, soc_d_mpc,meter_d, batt_output_d, soc_d = open_saved_data(path)
    cost_list_determ_1 = np.loadtxt(path + '/cost_list_determ_1.csv')
    cost_list_mpc_1 = np.loadtxt(path + '/cost_list_mpc_1.csv')

    #case 3

    #experiment_file = 'Code/Experiments/case3_1.json'
    #experiment_file = 'Code/Experiments/case3_7_homes.json'
    experiment_file = 'Code/Experiments/case3_9_homes.json'
    #experiment_file = 'Code/Experiments/case3_9_homes_2wks.json'


    #forecast 1
    path = '../Code/Results/case_3_9_homes_l2/f1'
    #path = 'Code/Results/case_3_9_homes_l2/two_weeks'
    mpc_problem = MPC(data, temp_data, experiment_file)
    # #to run case 3
    meter_d_mpc, batt_output_d_mpc, soc_d_mpc, cost_list_mpc_3, cost_list_3_no_bess = mpc_problem.deterministic_mpc(start_date, plotting=True, forecast=1)
    meter_d, batt_output_d, soc_d, cost_list_determ_3 = mpc_problem.full_deterministic(start_date)
    #mpc_problem.return_forecast(start_date+datetime.timedelta(days=1), plotting=True)
    np.savetxt(path + '/meter_d_mpc.csv', meter_d_mpc)
    np.savetxt(path + '/batt_output_d_mpc.csv', batt_output_d_mpc)
    np.savetxt(path + '/soc_d_mpc.csv', soc_d_mpc)
    np.savetxt(path + '/meter_d.csv', meter_d)
    np.savetxt(path + '/batt_output_d.csv', batt_output_d)
    np.savetxt(path + '/soc_d.csv', soc_d)
    np.savetxt(path + '/cost_list_mpc_3.csv', cost_list_mpc_3)
    np.savetxt(path + '/cost_list_3_no_bess.csv', cost_list_3_no_bess)
    np.savetxt(path + '/cost_list_determ_3.csv', cost_list_determ_3)


    meter_d_mpc, batt_output_d_mpc, soc_d_mpc,meter_d, batt_output_d, soc_d = open_saved_data(path)
    cost_list_mpc_3 = np.loadtxt(path + '/cost_list_mpc_3.csv')
    cost_list_3_no_bess = np.loadtxt(path + '/cost_list_3_no_bess.csv')
    cost_list_determ_3 = np.loadtxt(path + '/cost_list_determ_3.csv')

    plot_case_3(mpc_problem, meter_d_mpc, batt_output_d_mpc, soc_d_mpc,meter_d, batt_output_d, soc_d, start_date)

    print('PF cost savings')
    cost_savings(cost_list_3_no_bess, cost_list_determ_1, cost_list_determ_3)
    print('MPC cost savings')
    cost_savings(cost_list_3_no_bess, cost_list_mpc_1, cost_list_mpc_3)
    # #forecast 2
    # path = 'Code/Results/case_3_9_homes_l2/f2'
    # mpc_problem = MPC(data, temp_data, experiment_file)
    # #to run case 3
    # meter_d_mpc, batt_output_d_mpc, soc_d_mpc, cost_list_mpc_3, cost_list_3_no_bess = mpc_problem.deterministic_mpc(start_date, plotting=True, forecast=2)
    # meter_d, batt_output_d, soc_d, cost_list_determ_3 = mpc_problem.full_deterministic(start_date)
    # #mpc_problem.return_forecast(start_date+datetime.timedelta(days=1), plotting=True)
    # np.savetxt(path + '/meter_d_mpc.csv', meter_d_mpc)
    # np.savetxt(path + '/batt_output_d_mpc.csv', batt_output_d_mpc)
    # np.savetxt(path + '/soc_d_mpc.csv', soc_d_mpc)
    # np.savetxt(path + '/meter_d.csv', meter_d)
    # np.savetxt(path + '/batt_output_d.csv', batt_output_d)
    # np.savetxt(path + '/soc_d.csv', soc_d)
    # np.savetxt(path + '/cost_list_mpc_3.csv', cost_list_mpc_3)
    # np.savetxt(path + '/cost_list_3_no_bess.csv', cost_list_3_no_bess)
    # np.savetxt(path + '/cost_list_determ_3.csv', cost_list_determ_3)


    # meter_d_mpc, batt_output_d_mpc, soc_d_mpc,meter_d, batt_output_d, soc_d = open_saved_data(path)
    # cost_list_mpc_3 = np.append(cost_list_mpc_3, np.loadtxt(path + '/cost_list_mpc_3.csv'))
    # cost_list_3_no_bess = np.append(cost_list_3_no_bess, np.loadtxt(path + '/cost_list_3_no_bess.csv'))
    # cost_list_determ_3 = np.append(cost_list_determ_3, np.loadtxt(path + '/cost_list_determ_3.csv'))



    plot_case_1_3(mpc_problem, cost_list_3_no_bess, cost_list_mpc_1, cost_list_determ_1, cost_list_mpc_3, cost_list_determ_3)
    return
    

if __name__ == '__main__':
    main()

IndexError: index 0 is out of bounds for axis 0 with size 0